# AOC2019 - Day 7

Today's challenge: [Day 7](https://adventofcode.com/2019/day/7)

Data are manually downloaded into `./data/input_7.txt`

In [1]:
with open('./data/input_7.txt') as f:
    raw = f.read()

In [2]:
class Intcode:
    def __init__(self, xs, pos=0, i=None):
        self.xs = xs
        self.pos = pos
        self.i = i
        self.res = 0
        
    def display(self):
        return dict(enumerate(self.xs))
    
    @staticmethod
    def parse_op(x):
        params, op = divmod(x, 100)
        params, p1 = divmod(params, 10)
        p2 = params % 10
        return op, p1, p2
    
    def op1(self,i1,i2,out,ps):
        p1, p2 = ps
        i1_ = i1 if p1 == 1 else self.xs[i1]
        i2_ = i2 if p2 == 1 else self.xs[i2]
        self.xs[out] = i1_ + i2_
        return 4
    
    def op2(self,i1,i2,out,ps):
        p1, p2 = ps
        i1_ = i1 if p1 == 1 else self.xs[i1]
        i2_ = i2 if p2 == 1 else self.xs[i2]
        self.xs[out] = i1_ * i2_
        return 4

    def op3(self,x,i,pi):
        self.xs[i] = x
        return 2

    def op4(self,i,pi):
        #print("Output: ", self.xs[i])
        return (2, self.xs[i])
    
    def op5(self,i,out,ps):
        pi, pout = ps
        i_ = i if pi == 1 else self.xs[i]
        out_ = out if pout == 1 else self.xs[out]
        if i_ != 0:
            self.pos = out_
            return 0
        return 3
    
    def op6(self,i,out,ps):
        pi, pout = ps
        i_ = i if pi == 1 else self.xs[i]
        out_ = out if pout == 1 else self.xs[out]
        if i_ == 0:
            self.pos = out_
            return 0
        return 3
    
    def op7(self,i1,i2,out,ps):
        p1, p2 = ps
        i1_ = i1 if p1 == 1 else self.xs[i1]
        i2_ = i2 if p2 == 1 else self.xs[i2]
        self.xs[out] = 1 if i1_ < i2_ else 0
        return 4
    
    def op8(self,i1,i2,out,ps):
        p1, p2 = ps
        i1_ = i1 if p1 == 1 else self.xs[i1]
        i2_ = i2 if p2 == 1 else self.xs[i2]
        self.xs[out] = 1 if i1_ == i2_ else 0
        return 4
    
    def op99(self):
        return 0

    def run(self, x, set_mode=False):
        while True:
            op, *params = self.parse_op(self.xs[self.pos])
            if op%100 == 1:
                i1, i2, out = self.xs[self.pos+1:self.pos+4]
                self.pos += self.op1(i1, i2, out, params)
            elif op%100 == 2:
                i1, i2, out = self.xs[self.pos+1:self.pos+4]
                self.pos += self.op2(i1, i2, out, params)
            elif op%100 == 3:
                i = self.xs[self.pos+1]
                self.pos += self.op3(x, i, params)
                if set_mode:
                    return self
            elif op%100 == 4:
                i = self.xs[self.pos+1]
                step, res = self.op4(i, params)
                self.res = res
                self.pos += step
                return res, True
            elif op%100 == 5:
                i, out = self.xs[self.pos+1:self.pos+3]
                step = self.op5(i, out, params)
                self.pos += step
            elif op%100 == 6:
                i, out = self.xs[self.pos+1:self.pos+3]
                step = self.op6(i, out, params)
                self.pos += step
            elif op%100 == 7:
                i1, i2, out = self.xs[self.pos+1:self.pos+4]
                self.pos += self.op7(i1, i2, out, params)
            elif op%100 == 8:
                i1, i2, out = self.xs[self.pos+1:self.pos+4]
                self.pos += self.op8(i1, i2, out, params)
            elif op%100 == 99:
                self.op99()
                return self.res, False
            else:
                print("invalid instruction!", op)
                break

In [3]:
data = [int(x) for x in raw.strip().split(",")]

In [4]:
def amplify(xs):
    program1 = Intcode(data[:])
    program2 = Intcode(data[:])
    program3 = Intcode(data[:])
    program4 = Intcode(data[:])
    program5 = Intcode(data[:])
    programs = [Intcode(data[:]).run(m, True) for m in xs]
    res,_ = programs[0].run(0)
    res,_ = programs[1].run(res)
    res,_ = programs[2].run(res)
    res,_ = programs[3].run(res)
    return programs[4].run(res)[0]

In [5]:
from itertools import permutations

In [6]:
all_ = []
for ps in permutations(range(5)):
    res = amplify(ps)
    all_.append(res)
max(all_)

95757

In [7]:
#data = [3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5]
#data = [3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,
#        -5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,
#        53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10]
data = [int(x) for x in raw.strip().split(",")]

In [8]:
def mega_amplify(xs):
    step, res = 0, 0
    programs = [Intcode(data[:]).run(m, True) for m in xs]
    proceed = True
    while proceed:
        #print(step % 5, xs[0], res, programs[0].pos)
        res, proceed = programs[step%5].run(res)
        step += 1
    return programs[4].res

In [9]:
all_ = []
for ps in permutations(range(5,10)):
    res = mega_amplify(ps)
    all_.append(res)
max(all_)
#mega_amplify([9,8,7,6,5])

4275738